In [2]:
import pandas as pd

eicu_time = pd.read_csv("eicu_time_dataset.csv")
eicu_time.head()

,Unnamed: 0,patientunitstayid,hospitalid,time_window,death,ventilator,sepsis,acetamin,biotene,compazine,...,sex_is_male,sex_is_female,< 30,30 - 39,40 - 49,50 - 59,60 - 69,70 - 79,80 - 89,> 89
0,0,534765.0,148.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,534765.0,148.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,534765.0,148.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,534765.0,148.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,534765.0,148.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
eicu = pd.read_csv("eicu_dataset.csv")
eicu.head()

,patientunitstayid,hospitalid,death,acetamin,biotene,compazine,ferrous,imdur,lidocaine,milk of magnesia,...,sex_is_male,sex_is_female,< 30,30 - 39,40 - 49,50 - 59,60 - 69,70 - 79,80 - 89,> 89
0,141168.0,59.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,141178.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,141179.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,141194.0,73.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,141196.0,67.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
lab = pd.read_csv("data/eicu/lab.csv")

In [5]:
lab.head()

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.0,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.5,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.0,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.6,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.2,7.20,NaN,Units,2155


In [6]:
# key: lab_name, value: lab_name in eicu
lab_name_mapping = {
    "o2sat": "O2 Sat (%)",
    "pao2": "paO2",
    "paco2": "paCO2",
    "ph": "pH",
    "albu_lab": "albumin",
    "bands": "-bands",
    "bun": "BUN",
    "hct": "Hct",
    "inr": "PT - INR",
    "lactate": "lactate",
    "platelets": "platelets x 1000",
    "wbc": "WBC x 1000"
}

In [7]:
lab_name_mapping_values = list(lab_name_mapping.values())
unique_lab_names = lab['labname'].unique()
for value in lab_name_mapping_values:
    print(value, value in unique_lab_names)
    print()

O2 Sat (%) True

paO2 True

paCO2 True

pH True

albumin True

-bands True

BUN True

Hct True

PT - INR True

lactate True

platelets x 1000 True

WBC x 1000 True


In [41]:
selected_lab = lab[lab['labname'].isin(lab_name_mapping_values)].copy()

for value in lab_name_mapping_values:

    lab_values = selected_lab[selected_lab['labname'] == value]['labresult']
    
    # Filter out outliers using IQR
    Q1 = lab_values.quantile(0.25)
    Q3 = lab_values.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    lab_values_adjusted = lab_values.clip(lower_bound, upper_bound)
    
    mean_value = lab_values_adjusted.mean()
    std = lab_values_adjusted.std()
    
    # Normalize lab values using standard normal distribution
    lab_values_normalized = (lab_values_adjusted - mean_value) / std
    
    selected_lab.loc[selected_lab['labname'] == value, 'labresult'] = lab_values_normalized

-2.459458249051266e-15
-3.0263268761054807e-16
7.814527147778113e-17
1.985863469352039e-15
-1.9047329633462753e-15
1.122645695297145e-16
-2.7706974796293714e-16
1.1688769880382853e-15
-1.135523138431387e-15
-1.1321342501920443e-17
-7.205067530853485e-17
1.2528123367261733e-15


In [9]:
eicu_time.columns

Index(['Unnamed: 0', 'patientunitstayid', 'hospitalid', 'time_window', 'death',
       'ventilator', 'sepsis', 'acetamin', 'biotene', 'compazine',
       ...
       'sex_is_male', 'sex_is_female', '< 30', '30 - 39', '40 - 49', '50 - 59',
       '60 - 69', '70 - 79', '80 - 89', '> 89'],
      dtype='object', length=263)

In [20]:
eicu_columns = list(eicu_time.columns) + list(lab_name_mapping.keys())

In [25]:
lab

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,52307161,141168,2026,3,fibrinogen,177.00,177,mg/dL,mg/dL,2219
1,50363251,141168,1133,3,PT - INR,2.50,2.5,ratio,NaN,1208
2,49149139,141168,2026,1,magnesium,2.00,2.0,mg/dL,mg/dL,2090
3,50363250,141168,1133,3,PT,26.60,26.6,sec,sec,1208
4,66695374,141168,2141,7,pH,7.20,7.20,NaN,Units,2155
...,...,...,...,...,...,...,...,...,...,...
39132526,824772675,3353263,-7,3,WBC x 1000,6.40,6.4,K/mcL,K/uL,6
39132527,826470517,3353263,1733,3,RBC,4.67,4.67,M/mcL,M/uL,1774
39132528,824772678,3353263,-7,3,-monos,10.00,10,%,%,6
39132529,826470516,3353263,1733,3,WBC x 1000,6.60,6.6,K/mcL,K/uL,1774


In [47]:
from tqdm import tqdm

eicu_lab_time = pd.DataFrame(columns=eicu_columns)

bin_time = 12 * 60

for index, row in tqdm(eicu_time.iterrows(), total=eicu_time.shape[0]):
    
    eicu_row = pd.DataFrame(0.0, index=[0], columns=eicu_columns)
    
    # copy row values to eicu_row
    for column in eicu_time.columns:
        eicu_row[column] = row[column]
    
    # get patient lab data
    patient_id = row['patientunitstayid']
    lab_patient = selected_lab[selected_lab['patientunitstayid'] == patient_id]
    
    time_window = row['time_window']
    if time_window == -1:
        time_window_start = float('-inf')
        time_window_end = 0
    else:
        time_window_start = time_window * bin_time
        time_window_end = (time_window + 1) * bin_time
    
    lab_time_patient = lab_patient[(lab_patient['labresultoffset'] >= time_window_start) & (lab_patient['labresultoffset'] < time_window_end)]
    
    for lab_name, eicu_lab_name in lab_name_mapping.items():
        lab_value = lab_time_patient[lab_time_patient['labname'] == eicu_lab_name]['labresult']
        
        # average lab values
        if lab_value.shape[0] > 0:
            eicu_row[lab_name] = lab_value.mean()   
            
    eicu_lab_time = pd.concat([eicu_lab_time, eicu_row], ignore_index=True)

    if index == 10:
        break

eicu_lab_time.head()

  0%|          | 0/243285 [00:00<?, ?it/s]/var/folders/wd/d0j10lq906x4lfc0bphzybd80000gn/T/ipykernel_11036/1577728718.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eicu_lab_time = pd.concat([eicu_lab_time, eicu_row], ignore_index=True)
  0%|          | 10/243285 [00:00<5:38:21, 11.98it/s]


,Unnamed: 0,patientunitstayid,hospitalid,time_window,death,ventilator,sepsis,acetamin,biotene,compazine,...,paco2,ph,albumin,bands,bun,hct,inr,lactate,platelets,wbc
0,0.0,534765.0,148.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.222917,0.000980,0.014295,0.01,0.000675,0.298051,0.040441,0.043584,0.175507,0.034730
1,1.0,534765.0,148.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.216667,0.000974,0.000000,0.05,0.000747,0.265625,0.044118,0.048426,0.170116,0.037116
2,2.0,534765.0,148.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.0,534765.0,148.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4.0,534765.0,148.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
eicu_time_dataset = pd.read_csv("eicu_lab_time.csv")
eicu_time_dataset.head(20)

,Unnamed: 0,patientunitstayid,hospitalid,time_window,death,ventilator,sepsis,acetamin,biotene,compazine,...,paco2,ph,albu_lab,bands,bun,hct,inr,lactate,platelets,wbc
0,0.0,534765.0,148.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.222917,0.000980,0.014295,0.01,0.000675,0.298051,0.040441,0.043584,0.175507,0.034730
1,1.0,534765.0,148.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.216667,0.000974,0.000000,0.05,0.000747,0.265625,0.044118,0.048426,0.170116,0.037116
2,2.0,534765.0,148.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.0,534765.0,148.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4.0,534765.0,148.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5.0,534765.0,148.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,6.0,534765.0,148.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,7.0,534851.0,148.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.204167,0.000999,0.014295,0.00,0.001138,0.415323,0.055147,0.111380,0.065977,0.008248
8,8.0,534851.0,148.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.000000,0.000000,0.014945,0.00,0.001315,0.367944,0.055147,0.042373,0.055304,0.008722
9,9.0,534851.0,148.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [55]:
selected_lab[(selected_lab['patientunitstayid'] == 534765.0) & (selected_lab['labresultoffset'] >= 0) & (selected_lab['labresultoffset'] < 720) & (selected_lab['labname'] == 'platelets x 1000')]

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
5150409,172202064,534765,33,3,platelets x 1000,0.182083,563,K/mcL,K/MM3,47
5150506,173515014,534765,403,3,platelets x 1000,0.158150,489,K/mcL,K/MM3,419


array([148., 281.])